# SN_HW3

```Networks``` folder should be near this notebook

## Imports & Setup

In [25]:
import os
import json
import zipfile
from pathlib import Path
from typing import Dict, Tuple, List, Set, Optional
import random
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# plt.rcParams["figure.dpi"] = 100

# RANDOM_SEED = 42
# random.seed(RANDOM_SEED)
# np.random.seed(RANDOM_SEED)


BASE_DIR = Path(".").resolve()

OUTPUT_DIR = BASE_DIR / "outputs"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

NETWORKS_DIR = BASE_DIR / "Networks"


## Q1

## Locate

In [26]:
PART_A_DIR = NETWORKS_DIR / "Part_A"
OUTPUT_DIR_Q1=OUTPUT_DIR/'Q1'
OUTPUT_DIR_Q1.mkdir(parents=True, exist_ok=True)

print("Using NETWORKS_DIR =", NETWORKS_DIR)
print("Outputs will be written to:", OUTPUT_DIR_Q1)

Using NETWORKS_DIR = C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\Networks
Outputs will be written to: C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\outputs\Q1


## Helper Functions

In [27]:
def read_signed_undirected_csv(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    cols = [c.strip().lower() for c in df.columns]
    df.columns = cols
    if len(df.columns) < 3:
        raise ValueError(f"Expected >=3 columns in {path}, got {df.columns}")
    df = df.rename(columns={df.columns[0]: "u", df.columns[1]: "v", df.columns[2]: "sign"})
    return df[["u", "v", "sign"]].copy()

def read_directed_edge_csv(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    cols = [c.strip().lower() for c in df.columns]
    df.columns = cols
    if len(df.columns) < 2:
        raise ValueError(f"Expected >=2 columns in {path}, got {df.columns}")
    df = df.rename(columns={df.columns[0]: "src", df.columns[1]: "dst"})
    return df[["src", "dst"]].copy()

def ensure_int_nodes(df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
    out = df.copy()
    for c in cols:
        out[c] = out[c].astype(int)
    return out

def build_signed_graph_undirected(df_edges: pd.DataFrame, include_zero: bool=False) -> nx.Graph:
    G = nx.Graph()
    for u, v, s in df_edges[["u","v","sign"]].itertuples(index=False):
        u = int(u); v = int(v); s = int(s)
        if (not include_zero) and s == 0:
            continue
        G.add_edge(u, v, sign=s)
    return G

def save_df(df: pd.DataFrame, path: Path) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(path, index=False)
    print(f"***file saved*** => {path}")

def zip_outputs(folder: Path, zip_path: Path) -> None:
    with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
        for file_path in folder.rglob("*"):
            if file_path.is_file():
                zf.write(file_path, file_path.relative_to(folder.parent))

def deterministic_spring_layout(G: nx.Graph, seed: int=42):
    return nx.spring_layout(G, seed=seed)

def plot_signed_clusters(
    G: nx.Graph,
    node_cluster: Dict[int,int],
    out_path: Path,
    title: str = "",
    seed: int = 42
) -> None:
    pos = deterministic_spring_layout(G, seed=seed)
    clusters = sorted(set(node_cluster.values()))
    cluster_to_idx = {c:i for i,c in enumerate(clusters)}
    node_colors = [cluster_to_idx[node_cluster[n]] for n in G.nodes()]

    plt.figure(figsize=(10, 8))
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, cmap=plt.cm.tab20, node_size=250, alpha=0.9)

    pos_edges = [(u,v) for u,v,d in G.edges(data=True) if d.get("sign", 1) == 1]
    neg_edges = [(u,v) for u,v,d in G.edges(data=True) if d.get("sign", 1) == -1]

    nx.draw_networkx_edges(G, pos, edgelist=pos_edges, alpha=0.4)
    nx.draw_networkx_edges(G, pos, edgelist=neg_edges, style="dashed", alpha=0.4)
    nx.draw_networkx_labels(G, pos, font_size=8)
    plt.title(title)
    plt.axis("off")
    out_path.parent.mkdir(parents=True, exist_ok=True)
    plt.tight_layout()
    plt.savefig(out_path, dpi=200)
    # plt.show()
    plt.close()
    print(f"***file saved*** => {out_path}")


## Part 1 — Sign prediction in a balanced signed graph

In [28]:
part1_path = PART_A_DIR / "1" / "balanced_graph.csv"
df1 = read_signed_undirected_csv(part1_path)
df1 = ensure_int_nodes(df1, ["u","v"])
df1["sign"] = df1["sign"].astype(int)

df_known = df1[df1["sign"] != 0].copy()
df_unknown = df1[df1["sign"] == 0].copy()

G_known = build_signed_graph_undirected(df_known, include_zero=False)

labels: Dict[int,int] = {}

def assign_labels_via_bfs(G: nx.Graph) -> Dict[int,int]:
    labels: Dict[int,int] = {}
    for start in G.nodes():
        if start in labels:
            continue
        labels[start] = 0
        queue = [start]
        while queue:
            u = queue.pop(0)
            for v, data in G[u].items():
                s = int(data.get("sign", 1))
                desired = labels[u] ^ (1 if s == -1 else 0)
                # print(f's={s},labels[u]={labels[u]} ,(1 if s == -1 else 0)={(1 if s == -1 else 0)},desired={desired}')
                if v not in labels:
                    labels[v] = desired
                    queue.append(v)
                else:
                    if labels[v] != desired:
                        raise ValueError(
                            f"Inconsistent constraints encountered at edge ({u},{v}) with sign {s}."
                        )
    return labels

labels = assign_labels_via_bfs(G_known)

all_nodes = set(df1["u"]).union(set(df1["v"]))
for n in all_nodes:
    if int(n) not in labels:
        labels[int(n)] = 0

pred_rows = []
for u, v in df_unknown[["u","v"]].itertuples(index=False):
    u = int(u); v = int(v)
    pred = 1 if labels[u] == labels[v] else -1
    pred_rows.append((u, v, pred))

df_pred = pd.DataFrame(pred_rows, columns=["u","v","predicted_sign"])

df_partition = pd.DataFrame(sorted(labels.items()), columns=["node","group"])
save_df(df_partition, OUTPUT_DIR_Q1 / "q1_part1_partition.csv")
save_df(df_pred, OUTPUT_DIR_Q1 / "q1_part1_predicted_unknown_edges.csv")

print("Part 1 done.")
print("number of node:",len(all_nodes) ,"Known edges:", len(df_known), "Unknown edges:", len(df_unknown))
print(f"group'0'={(df_partition['group']==0).sum()} , group'1'={(df_partition['group']==1).sum()}")
print(f"predicted_sign'1'={(df_pred['predicted_sign']==1).sum()} , predicted_sign'-1'={(df_pred['predicted_sign']==-1).sum()}")

***file saved*** => C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\outputs\Q1\q1_part1_partition.csv
***file saved*** => C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\outputs\Q1\q1_part1_predicted_unknown_edges.csv
Part 1 done.
number of node: 50 Known edges: 135 Unknown edges: 89
group'0'=32 , group'1'=18
predicted_sign'1'=40 , predicted_sign'-1'=49


## Part 2 — Balance test via Super-node reduction

In [29]:
part2_dir = PART_A_DIR / "2"
network_files = sorted(part2_dir.glob("network_*.csv"))

def union_find_components(edges: List[Tuple[int,int]], nodes: Set[int]) -> Dict[int,int]:
    parent = {n:n for n in nodes}
    rank = {n:0 for n in nodes}

    def find(x):
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x

    def union(a,b):
        ra, rb = find(a), find(b)
        if ra == rb:
            return
        if rank[ra] < rank[rb]:
            parent[ra] = rb
        elif rank[ra] > rank[rb]:
            parent[rb] = ra
        else:
            parent[rb] = ra
            rank[ra] += 1

    for u,v in edges:
        union(u,v)

    roots = {find(n) for n in nodes}
    root_to_sid = {r:i for i,r in enumerate(sorted(roots))}
    node_to_sid = {n: root_to_sid[find(n)] for n in nodes}
    return node_to_sid

def supernode_balance_test(df: pd.DataFrame):
    df = ensure_int_nodes(df, ["u","v"])
    df["sign"] = df["sign"].astype(int)
    nodes = set(df["u"]).union(set(df["v"]))

    pos_edges = [(int(u),int(v)) for u,v,s in df[["u","v","sign"]].itertuples(index=False) if int(s)==1]
    neg_edges = [(int(u),int(v)) for u,v,s in df[["u","v","sign"]].itertuples(index=False) if int(s)==-1]

    node_to_super = union_find_components(pos_edges, nodes)

    for u,v in neg_edges:
        if node_to_super[u] == node_to_super[v]:
            return {
                "balanced": False,
                "reason": "negative_edge_inside_supernode",
                "example": (u,v),
                "node_to_super": node_to_super,
                "reduced_graph": None,
                "odd_cycle": None
            }

    H = nx.Graph()
    for sid in set(node_to_super.values()):
        H.add_node(sid)
    for u,v in neg_edges:
        su, sv = node_to_super[u], node_to_super[v]
        if su != sv:
            H.add_edge(su, sv, sign=-1)

    if not nx.is_bipartite(H):
        odd = None
        try:
            cycles = nx.cycle_basis(H)
            for c in cycles:
                if len(c) % 2 == 1:
                    odd = c
                    break
        except Exception:
            odd = None
        return {
            "balanced": False,
            "reason": "reduced_graph_not_bipartite",
            "example": None,
            "node_to_super": node_to_super,
            "reduced_graph": H,
            "odd_cycle": odd
        }

    return {
        "balanced": True,
        "reason": "ok",
        "example": None,
        "node_to_super": node_to_super,
        "reduced_graph": H,
        "odd_cycle": None
    }

summary_rows = []

for f in network_files:
    df = read_signed_undirected_csv(f)
    result = supernode_balance_test(df)
    name = f.stem

    assign_df = pd.DataFrame(sorted(result["node_to_super"].items()), columns=["node","supernode"])
    save_df(assign_df, OUTPUT_DIR_Q1 / f"q1_part2_supernode_assignment_{name}.csv")

    if result["reduced_graph"] is not None:
        H = result["reduced_graph"]
        red_df = pd.DataFrame([(u,v) for u,v in H.edges()], columns=["super_u","super_v"])
    else:
        red_df = pd.DataFrame(columns=["super_u","super_v"])
    save_df(red_df, OUTPUT_DIR_Q1 / f"q1_part2_reduced_graph_edges_{name}.csv")

    nodes = set(df["u"]).union(set(df["v"]))
    summary_rows.append({
        "network": name,
        "balanced": bool(result["balanced"]),
        "reason": result["reason"],
        "example_edge": str(result.get("example")),
        "odd_cycle_supernodes": str(result.get("odd_cycle")),
        "num_nodes": int(len(nodes)),
        "num_edges": int(len(df)),
        "num_supernodes": int(len(set(result["node_to_super"].values()))),
    })

df_summary = pd.DataFrame(summary_rows)

save_df(df_summary, OUTPUT_DIR_Q1 / "q1_part2_balance_summary.csv")

print("Part 2 done. Summary saved to outputs/q1_part2_balance_summary.csv")

***file saved*** => C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\outputs\Q1\q1_part2_supernode_assignment_network_a.csv
***file saved*** => C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\outputs\Q1\q1_part2_reduced_graph_edges_network_a.csv
***file saved*** => C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\outputs\Q1\q1_part2_supernode_assignment_network_b.csv
***file saved*** => C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\outputs\Q1\q1_part2_reduced_graph_edges_network_b.csv
***file saved*** => C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\outputs\Q1\q1_part2_supernode_assignment_network_c.csv
***file saved*** => C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\outputs\Q1\q1_part2_reduced_graph_edges_network_c.csv
***file saved*** => C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\outputs\

## Part 3 — Clusterability (Weak balance) and visualization

In [30]:
part3_dir = PART_A_DIR / "3"
files3 = sorted(part3_dir.glob("network_*.csv"))

def positive_components_clustering(df: pd.DataFrame):
    df = ensure_int_nodes(df, ["u","v"])
    df["sign"] = df["sign"].astype(int)
    nodes = set(df["u"]).union(set(df["v"]))

    Gpos = nx.Graph()
    Gpos.add_nodes_from(nodes)
    for u,v,s in df[["u","v","sign"]].itertuples(index=False):
        if int(s) == 1:
            Gpos.add_edge(int(u), int(v))

    comps = list(nx.connected_components(Gpos))
    node_to_cluster = {}
    for cid, comp in enumerate(comps):
        for n in comp:
            node_to_cluster[int(n)] = cid

    for u,v,s in df[["u","v","sign"]].itertuples(index=False):
        if int(s) == -1 and node_to_cluster[int(u)] == node_to_cluster[int(v)]:
            return node_to_cluster, False, (int(u),int(v))

    return node_to_cluster, True, None

summary3 = []

for f in files3:
    name = f.stem
    df = read_signed_undirected_csv(f)
    node_to_cluster, ok, example = positive_components_clustering(df)

    assign_df = pd.DataFrame(sorted(node_to_cluster.items()), columns=["node","cluster"])
    save_df(assign_df, OUTPUT_DIR_Q1 / f"q1_part3_cluster_assignment_{name}.csv")

    sizes = pd.Series(list(node_to_cluster.values())).value_counts().sort_index()
    size_list = [int(sizes[i]) for i in sizes.index]

    G = build_signed_graph_undirected(df, include_zero=False)
    fig_path = OUTPUT_DIR_Q1 / f"q1_part3_clusters_{name}.png"
    plot_signed_clusters(G, node_to_cluster, fig_path, title=f"{name} clusters (positive components)")

    summary3.append({
        "network": name,
        "weakly_balanced_check": bool(ok),
        "violating_negative_edge": str(example),
        "num_clusters": int(len(set(node_to_cluster.values()))),
        "cluster_sizes": str(size_list),
        "figure_path": str(fig_path.relative_to(BASE_DIR))
    })

df3 = pd.DataFrame(summary3)
save_df(df3, OUTPUT_DIR_Q1 / "q1_part3_clusterability_summary.csv")

print("Part 3 done. Summary saved to outputs/q1_part3_clusterability_summary.csv")

***file saved*** => C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\outputs\Q1\q1_part3_cluster_assignment_network_a.csv
***file saved*** => C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\outputs\Q1\q1_part3_clusters_network_a.png
***file saved*** => C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\outputs\Q1\q1_part3_cluster_assignment_network_b.csv
***file saved*** => C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\outputs\Q1\q1_part3_clusters_network_b.png
***file saved*** => C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\outputs\Q1\q1_part3_cluster_assignment_network_c.csv
***file saved*** => C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\outputs\Q1\q1_part3_clusters_network_c.png
***file saved*** => C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\outputs\Q1\q1_part3_cluster_assignment_network_

## Part 4 — Line Index for a 4-cluster signed network (random + heuristic)

In [31]:
part4_path = PART_A_DIR / "4" / "network_line_index.csv"
df4 = read_signed_undirected_csv(part4_path)
df4 = ensure_int_nodes(df4, ["u","v"])
df4["sign"] = df4["sign"].astype(int)

nodes4 = sorted(set(df4["u"]).union(set(df4["v"])))
k = 4
alpha = 0.5

def compute_PN_line_index(df: pd.DataFrame, assignment: Dict[int,int], alpha: float = 0.5) -> Tuple[int,int,float]:
    P = 0
    N = 0
    for u,v,s in df[["u","v","sign"]].itertuples(index=False):
        u = int(u); v = int(v); s = int(s)
        cu, cv = assignment[u], assignment[v]
        if s == 1 and cu != cv:
            P += 1
        elif s == -1 and cu == cv:
            N += 1
    LI = alpha * P + (1 - alpha) * N
    return P, N, float(LI)

rng = np.random.default_rng(42)
assign_random = {int(n): int(rng.integers(0, k)) for n in nodes4}
P_r, N_r, LI_r = compute_PN_line_index(df4, assign_random, alpha=alpha)

save_df(pd.DataFrame(sorted(assign_random.items()), columns=["node","cluster"]),
        OUTPUT_DIR_Q1 / "q1_part4_random_assignment.csv")

adj = {int(n): [] for n in nodes4}
for u,v,s in df4[["u","v","sign"]].itertuples(index=False):
    u=int(u); v=int(v); s=int(s)
    adj[u].append((v,s))
    adj[v].append((u,s))

def cost_P_plus_N(df: pd.DataFrame, assignment: Dict[int,int]) -> int:
    P, N, _ = compute_PN_line_index(df, assignment, alpha=0.5)
    return P + N

def delta_move(node: int, new_c: int, assignment: Dict[int,int]) -> int:
    old_c = assignment[node]
    if new_c == old_c:
        return 0
    delta = 0
    for nei, s in adj[node]:
        c_nei = assignment[nei]
        cur_bad = 1 if (s == 1 and old_c != c_nei) or (s == -1 and old_c == c_nei) else 0
        new_bad = 1 if (s == 1 and new_c != c_nei) or (s == -1 and new_c == c_nei) else 0
        delta += (new_bad - cur_bad)
    return delta

def local_search(initial: Dict[int,int], max_iters: int = 20000) -> Tuple[Dict[int,int], List[int]]:
    assign = dict(initial)
    history = [cost_P_plus_N(df4, assign)]
    improved = True
    it = 0
    while improved and it < max_iters:
        improved = False
        it += 1
        for n in nodes4:
            best_delta = 0
            best_c = assign[n]
            for c in range(k):
                d = delta_move(n, c, assign)
                if d < best_delta:
                    best_delta = d
                    best_c = c
            if best_delta < 0:
                assign[n] = best_c
                history.append(history[-1] + best_delta)
                improved = True
    return assign, history

best_assign = None
best_cost = None
best_history = None

num_restarts = 20
rng2 = np.random.default_rng(123)

for _ in range(num_restarts):
    init = {int(n): int(rng2.integers(0, k)) for n in nodes4}
    sol, hist = local_search(init)
    c = hist[-1]
    if best_cost is None or c < best_cost:
        best_cost = c
        best_assign = sol
        best_history = hist

P_h, N_h, LI_h = compute_PN_line_index(df4, best_assign, alpha=alpha)

save_df(pd.DataFrame(sorted(best_assign.items()), columns=["node","cluster"]),
        OUTPUT_DIR_Q1 / "q1_part4_heuristic_assignment.csv")

plt.figure(figsize=(7,4))
plt.plot(best_history)
plt.xlabel("Improvement step")
plt.ylabel("P+N (misplaced edges)")
plt.title("Local search history (best restart)")
plt.tight_layout()
history_path = OUTPUT_DIR_Q1 / "q1_part4_local_search_history.png"
plt.savefig(history_path, dpi=200)
print(f"***file saved*** => {history_path}")
# plt.show()
plt.close()

print("Part 4 done.")
print("Random: P,N,LI=", P_r, N_r, LI_r)
print("Heuristic best: P,N,LI=", P_h, N_h, LI_h)


***file saved*** => C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\outputs\Q1\q1_part4_random_assignment.csv
***file saved*** => C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\outputs\Q1\q1_part4_heuristic_assignment.csv
***file saved*** => C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\outputs\Q1\q1_part4_local_search_history.png
Part 4 done.
Random: P,N,LI= 82 17 49.5
Heuristic best: P,N,LI= 0 0 0.0


## Part 5 — Transitivity in a directed network

In [32]:
part5_path = PART_A_DIR / "5" / "network_transitivity.csv"
df5 = read_directed_edge_csv(part5_path)
df5 = ensure_int_nodes(df5, ["src","dst"])

Gd = nx.DiGraph()
for s,t in df5[["src","dst"]].itertuples(index=False):
    Gd.add_edge(int(s), int(t))

edges_set: Set[Tuple[int,int]] = set(Gd.edges())

two_step_total = 0
transitive_triples = 0
missing_edges_2step: Set[Tuple[int,int]] = set()

out_neighbors = {u: list(Gd.successors(u)) for u in Gd.nodes()}

for i in Gd.nodes():
    for j in out_neighbors.get(i, []):
        for k in out_neighbors.get(j, []):
            two_step_total += 1
            if (i, k) in edges_set:
                transitive_triples += 1
            else:
                missing_edges_2step.add((i, k))

transitivity_ratio = (transitive_triples / two_step_total) if two_step_total > 0 else float("nan")

closure = nx.transitive_closure(Gd)
closure_edges = set(closure.edges())
edges_to_add_full = sorted(list(closure_edges - edges_set))

missing_2step_df = pd.DataFrame(sorted(list(missing_edges_2step)), columns=["src","dst"])
save_df(missing_2step_df, OUTPUT_DIR_Q1 / "q1_part5_missing_edges_for_2step_paths.csv")

add_df = pd.DataFrame(edges_to_add_full, columns=["src","dst"])
save_df(add_df, OUTPUT_DIR_Q1 / "q1_part5_edges_to_add_for_transitivity.csv")

print("Part 5 done.")
print("Gd.nodes():",len(Gd.nodes()))
print("2-step paths:", two_step_total)
print("transitive triples:", transitive_triples)
print("ratio:", transitivity_ratio)
print("edges to add for full transitivity:", len(edges_to_add_full))

***file saved*** => C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\outputs\Q1\q1_part5_missing_edges_for_2step_paths.csv
***file saved*** => C:\programing\University of Tehran\social-netwok-8101927\HW\SN_HW3\code\Q1\outputs\Q1\q1_part5_edges_to_add_for_transitivity.csv
Part 5 done.
Gd.nodes(): 200
2-step paths: 3299
transitive triples: 70
ratio: 0.021218551076083662
edges to add for full transitivity: 38186
